<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#第５章　Step-4.-ファーマコフォアスクリーニング" data-toc-modified-id="第５章　Step-4.-ファーマコフォアスクリーニング-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>第５章　Step 4. ファーマコフォアスクリーニング</a></span></li></ul></div>

# 第５章　Step 4. ファーマコフォアスクリーニング

第５章ではファーマコフォアスクリーニングを実施しています。ファーマコフォアとして２種類の組み合わせをもちいて、２段階のスクリーニングを実施しました。

いずれもファーマコフォアの選定には共結晶構造を利用していますが、１段階目では

* 複数の共結晶構造から**PLIF**を作成して、**タンパク質側**の相互作用に重要な残基を推定　（第５章１節）
* 相互作用に重要と思われる**リガンド側**の特徴を抜き出し、ファーマコフォアを作成
* **既知活性化合物群**に対して上記ファーマコフォアによる検索を実施し処理方法を確認　（第５章３節）
* **スクリーニング対象化合物群**に対して実際に適用　（第５章４節）

という手順で実施しています。また、RDKitをもちいてファーマコフォアスクリーニングを実施するための予備調査として、認識される特徴構造（フィーチャー）の定義を確認しています（第５章２節）

ファーマコフォアスクリーニング　２段階目ではファーマコフォアの選定基準をより明確にするため、

* 複数の共結晶構造を重ね合わせ、残基とリガンドとの相互作用を確認
* 既知活性化合物のSAR（伸長による疎水性ポケットの利用）を踏まえたファーマコフォアの選定　（第５章４節）

に取り組みました。


具体的な処理としては以下の操作となります。

（１）１段階目

**Protein-Ligand Interaction Profiler(PLIP)**による相互作用解析結果を、残基ごとに相互作用の有無をビット（0 or 1）で表すことで**PLIF**の模倣を試みました。PandasのDataFrameを用いてビットをヒートマップで表した場合が以下のような図となります。

```python
%matplotlib inline
sns.heatmap(FP_df_num)
```

<img src="./images/5-1_image9.png">

（横軸：残基、縦軸：共結晶構造のPDB ID（最下段はビットの合計））



ここから「PDB id:5NIX(Ligand:8YQ)」を例に重要と思われる相互作用を以下のように推定しました。

||残基番号|アミノ酸|距離|相互作用||残基番号|アミノ酸|距離|相互作用|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|8YQ</br>(chain A/B)|||||8YQ</br>(chain C/D)|54C|ILE|3.89|Hydrophobic|
||56B|TYR|3.82|Hydrophobic||56C|TYR|3.42/3.62|Hydrophobic|
||66B|GLN|3.72|Hydrophobic||||||
||115A/B|MET|3.70/3.80|Hydrophobic||115C/D|MET|3.85/3.82|Hydrophobic|
||121A/B|ALA|3.69/3.75|Hydrophobic||121C/D|ALA|3.96/3.57|Hydrophobic|
||123A|TYR|3.80/3.52/3.85|Hydrophobic||123D|TYR|3.72|Hydrophobic|
||124A|LYS|5.03/3.18|π-cation/Salt Bridges||124D|Lys|3.70|Water Bridges/Salt Bridges|



PyMolを利用して上記相互作用を確認し、３点の特徴構造（フィーチャー）を選択、互いの位置関係（距離、角度）からファーマコフォアを作成しました。

<img src="./images/5-3_image13.png">


RDKitを用いたファーマコフォアスクリーニング手順の流れ（①〜⑤）は以下となります。

① フィーチャーの設定

* ChemicalFeatures.FreeChemicalFeature(**Feature Family**, **Feature Type**, **位置**) 
* 位置の指定はGeometry.Point3D(**x座標**, **y座標**, **z座標**)

```python
# Aromaticフィーチャーの場合
feat_1=ChemicalFeatures.FreeChemicalFeature('Aromatic',Geometry.Point3D(-7.625, 10.407, -21.653))
# feat_2、feat_3も同様に設定 
feats = [feat_1,feat_2,feat_3]
```

② ファーマコフォアの設定

設定した複数のフィーチャーの組み合わせ（リストオブジェクト）に対して、互いの位置関係（距離情報）を設定します。この時、距離は範囲（上限、下限を設定した許容範囲）として指定することができます。

```python
pcophore = Pharmacophore.Pharmacophore(feats) # ファーマコフォアの設定
d_upper = 1.5  # 特性基間の距離の許容範囲(上限値)
d_lower = 0.5 # 特性基間の距離の許容範囲(下限値)

# feat_1とfeat_2の距離 6.3Åを基準に、下限と上限を設定
pcophore.setLowerBound(0,1, 6.3-d_lower)
pcophore.setUpperBound(0,1, 6.3+d_upper)

# feat_2とfeat_3、feat_1とfeat_3の関係性も同様に定義
```

③ フィーチャーの有無でのスクリーニング

EmbedLib.MatchPharmacophoreToMolを使うことで対象の化合物に対してファーマコフォアのマッピングを行い、「全てのフィーチャーが見つけられたか否か（True / False）」、「Trueの場合、見つけられたフィーチャーのリスト」を取得します。

```python
# test_mHは対象の分子のmolオブジェクト
match, mList = EmbedLib.MatchPharmacophoreToMol(test_mH, fdef, pcophore)
```

④　距離条件を用いたスクリーニング

rdDistGeom.GetMoleculeBoundsMatrixで対象化合物の距離行列を取得し、Parm3D.EmbedLib.GetAllPharmacophoreMatchesでファーマコフォアで定義されている距離の条件を満たすか否かを判定します。

```python
# 距離行列の取得
bounds = rdDistGeom.GetMoleculeBoundsMatrix(test_mH)
#　ファーマコフォアの距離条件を満たす組み合わせの取得 
pList =EmbedLib.GetAllPharmacophoreMatches(mList,bounds,pcophore) 
```

⑤　距離条件を満たす３次元構造を発生させられるか否かを判断基準とした判定

ファーマコフォアの条件満たす原子IDのリストを取得し、Pharm3D.EmbedLib.EmbedPharmacophoreで距離情報を拘束条件にした3D構造を発生させます。構造発生の可否（発生数）を基準としてスクリーニングを実施しました。

```python
Generated_embeds = []
for phMatch in phMatches:
    bm,embeds,nFail =EmbedLib.EmbedPharmacophore(test_mH, phMatch,
                                                 pcophore,count=20,
                                                 silent=1)
    Generated_embeds.append(len(embeds))

#発生数の合計値「sum(Generated_embeds)」を判定基準として使用
```

以上がファーマコフォアスクリーニングの流れですが、注意点として今回はエラーの回避のため、不斉点の情報を無視して実行しています。（スクリーニング前に一度SMILESに変換してから、２次元に再度戻すという処理を加えています）

```python
smi = Chem.MolToSmiles(mol, isomericSmiles=False)
mol_re = Chem.MolFromSmiles(smi)
AllChem.Compute2DCoords(mol_re)
```


（２） ２段階目

ファーマコフォアスクリーニングの手順としては（１）と同様ですが、PyMolで共結晶構造を重ね合わせた結果から、以下のファーマコフォアを選定して実施しました。

<img src="./images/5-5_image16.png">



以上、２段階のファーマコフォアスクリーニングを実施し、部分構造による絞り込み(Step 3)と合わせて494個の化合物に絞り込みました。